# Dask Bag - CMIP6 Indexer Analysis

We demonstrate how to use Dask.bag to read and process semi-structured data such as JSON.

* load JSON files
* Parse JSON object as dictionary
* Apply map, filter, group
---

- Authors: NCI Virtual Research Environment Team
- Keywords: Dask bag, JSON 
- Creation Date: 2020-Sep
- Lineage/Reference: This tutorial is derived from the [dask tutorial](https://github.com/dask/dask-tutorial).
---

## About Dask.bag

`Dask.bag` excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution

## Setup

Choose from the following two options to create a client:

In [ ]:
# If you run this notebook on your local computer or NCI's VDI instance, you can create cluster
from dask.distributed import Client
client = Client()
print(client)

In [ ]:
# If you run this notebook on Gadi under pangeo environment, you can create cluster using scheduler.json file
from dask.distributed import Client, LocalCluster
client = Client(scheduler_file='scheduler.json')
print(client)

<div class="alert alert-info">
<b>Warning: Please make sure you specify the correct path to the scheduler.json file within your environment.</b>  
</div>

Starting the Dask Client will provide a dashboard which is useful to gain insight into the computation. The link to the dashboard will become visible when you create the Client. We recommend having the Client open on one side of your screen and your notebook open on the other side, which will be useful for learning purposes.

## Creation

You can create a `Bag` from a Python sequence, from files, or from data on a remote service URL, etc. We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

The data are partitioned into blocks, and there are many items per block.

In the first example, there are two partitions, each containing five elements:

In [ ]:
# each element is an integer
import dask.bag as db
b1 = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b1.take(3)

In the second example, dask bag sequences a list with different data types:

In [ ]:
nested_containers = [[0, 1, 2, 3],{},[6.5, 3.14], 'Python', {'version':3}, '' ]
b2 = db.from_sequence(nested_containers) 
b2.count().compute()

In the third example, each file is partitioned into one or more bytes blocks:

In [ ]:
# Each element is a text file, where each line is a JSON object
import os
b3 = db.read_text(os.path.join('/g/data/dk92/notebooks/demo_data/cmip6_json','*.json'))
b3.take(1)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.

In [ ]:
# for each even number, calculate the power of 2 of this number
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

You can either convert the `dask.bag` into a list directly, or by calling the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [ ]:
list(c)

In [ ]:
d=c.compute()
d

### CMIP6 Indexer data example

This example's data are global attributes (metadata) from some CMIP6 climate data. Each file is a JSON encoded dictionary with a list of keys, which are basically CMIP6 vocabularies like:

*  Conventions: CF-1.7 CMIP-6.2
*  activity_id: CMIP
*  creation_date: 2019-11-15T02:37:21Z
*  experiment: 1 percent per year increase in CO2
*  ...

JSON (JavaScript Object Notation) data files have the following features:

- stored as plain text
- common web format
- direct mapping to Python lists & dictionaries



### Data preprocessing 

`Dask.bag` only takes double quotes in the JSON object. Therefore, we need to replace the single quotes with double quotes in the JSON files. 

Using Python's `glob` module to search all the historical model files (prepared for this exercise):

In [ ]:
import glob
files = glob.glob('/g/data/dk92/notebooks/demo_data/cmip6_json/*.json')

In [ ]:
len(files)

Use the `json` python module to read a JSON file:

In [ ]:
import json
with open(files[0]) as f:
    items = json.load(f)
type(items)

In [ ]:
items['mip_era']

Convert the JSON files into a Dask `bag`. Note that how we load the data determines its type.

In [ ]:
import dask.bag as db
items = db.read_text(files[0]) 
items.take(1) # Note: tuple containing a string, but we want a dictionary

In [ ]:
import dask.bag as db
items = db.read_text(files[0]).map(json.loads)
items.take(1) # Note: tuple containing a dictionary

### Basic Queries

Once we parse our JSON data into useful Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [ ]:
js = db.read_text('/g/data/dk92/notebooks/demo_data/cmip6_json/*.json').map(json.loads)
js

In [ ]:
# Re-partition the bag so that `.take()` will return the number of elements you want.
js1 = js.repartition(1)
js1.take(3)

List all the variables using `.map()`:

In [ ]:
js.map(lambda record: record['variable_id']).compute()

Find out how many files with variable `cli` using `.filter()`:

In [ ]:
specific_item = js.filter(lambda element: element['variable_id'] == 'cli')
specific_item.count().compute()

Find out how many data files have the nominal resolution of 250 km.

In [ ]:
d_250km = js.filter(lambda element: element['nominal_resolution'] == '250 km').count().compute()
d_250km

It is common to do many of these steps in one pipeline, only calling `compute` or `take` at the end.

In [ ]:
result = (js1.filter(lambda element: element['variable_id'] == 'cli')
           .map(lambda element: element['tracking_id']))
result

In [ ]:
result.take(5)

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [ ]:
result = js.groupby(lambda item: item['institution_id'])
result

In [ ]:
%%time
result = js.groupby(lambda item: item['institution_id']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction for each group
3.  A combined binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

The reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

We find the number of files with the same institute name.

In [ ]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='institution_id', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  DataFrames are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [ ]:
df = js.to_dataframe()
df

This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our previous computation of numbers of files with the same institute name?  It turns out that `dask.dataframe.groupby()` is faster than `dask.bag.groupby()` ; but it still cannot match `dask.bag.foldby()` in this case.

In [ ]:
df.groupby('institution_id')

In [ ]:
%time df.groupby('institution_id').count().compute()

## Limitations

Bags provide very general computation (any Python function). This generality comes at cost. Bags have the following known limitations:

1.  Bag operations tend to be slower than array/dataframe computations in the same way that Python tends to be slower than NumPy/Pandas.
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible. Using ``Bag.foldby`` requires more thought. Even better, consider creating a normalised `dataframe`.

## Close client

Remember to close the dask client when done.

In [ ]:
client.close()